## Project 2: Feature Detection and Matching
### Gabriel Hofer
### CSC-414 Introduction to Computer Vision
### Instructor: Dr. Randy C. Hoover



### Gaussian Kernel
param r - number of rows  
param c - number of columns  
param s - sigma  
returns 2d numpy array, the gaussian kernel  

In [ ]:
import numpy as np
import math
def gk(r,c,s):
	arr=np.zeros((r,c))
	for i in range(r):
		for j in range(c):
			arr[i,j]=(1/(2*math.pi*s*s))*math.exp(-1*((i-r//2)*(i-r//2)+(j-c//2)*(j-c//2))/(2*s*s))
	return arr


### Harris Corner Detector  
Harris Corner Detection algorithm  
:param img - image, numpy array  
:param g - square guassian matrix, w  
:th - threshold value for R  
:returns - list of pairs. each pair is coordinates of an interst point, (x,y)  

In [ ]:
import numpy as np
import scipy.signal as ss

xsobel = np.array([
    [1,0,-1],
    [2,0,-2],
    [1,0,-1]])

ysobel = np.array([
    [1,2,1],
    [0,0,0],
    [-1,-2,-1]])

def harris(img,g,th):
    n=g.shape[0]
    x=ss.convolve2d(img,xsobel)
    y=ss.convolve2d(img,ysobel)    
    xx=x*x
    yy=y*y
    xy=x*y
    r,c=img.shape[0],img.shape[1]
    avgR=0
    R=125
    feat=[]
    for i in range(r-n):
        for j in range(c-n):
            sxx=(g*xx[i:i+n, j:j+n]).sum()
            syy=(g*yy[i:i+n, j:j+n]).sum()
            sxy=(g*xy[i:i+n, j:j+n]).sum()
            d=(sxx*syy)-(sxy*sxy) 
            t=sxx+syy
            if t!=0:
                R = d/t
            avgR+=R
            if R>th: 
                feat.append([i,j])
    avgR/=r*c
    print("average r: "+str(avgR))
    return feat



### Driver Code for Project 

In [ ]:
"""
Author: Gabriel Hofer
Course: CSC-414
Instructor: Dr. Randy Hoover
"""
from gk import gk
from harris import harris
from showDots import showDots
import numpy as np
import sys
from skimage import io
from skimage.color import rgb2gray

from showFeatures import showFeatures
from sift import printH, makeH, sift

# read image, convert to gray
img = io.imread(sys.argv[1]);
cpy = img
print("orig shape: "+str(img.shape))
gry = rgb2gray(img)
print("gray shape: "+str(gry.shape))

# get list of locations of interest points
loc = harris(gry,gk(3,3,1),1)
print("number of features: "+str(len(loc)))

# make sift descriptors
D = sift(gry,loc)
print("number of descriptors: "+str(len(D)))

# showFeatures(img,loc,D)

from matching import matchSIFT
M = matchSIFT(D,D,40,40)

import cv2
from drawMatches import drawMatches
img2 = drawMatches(img,loc,img,loc,M)

io.imshow( img2,vmin=0,vmax=255,cmap="gray")
io.show()

#io.imshow((img * 255).astype(np.uint8)  , vmin=0, vmax=255, cmap="gray")
#io.show()
#io.imsave("out.png",img)





### SIFT Descriptor

In [ ]:
import numpy as np
from gk import gk
import math


def printH(H):
    for i in H: print(str(i)+' ',end='')
    print()

def makeH(W):
    H=np.zeros(8)
    for i in range(1,5):
        for j in range(1,5):
            m = math.sqrt((W[i+1,j]-W[i-1,j])**2 + (W[i,j+1]-W[i,j-1])**2)
            if W[i,j+1] != W[i,j-1] and W[i+1,j] != W[i-1,j]: 
                theta = math.atan2((W[i+1,j]-W[i-1,j]), (W[i,j+1]-W[i,j-1]))
            else: continue
            norm=int(math.floor((theta+math.pi)*8/(2*math.pi))%8)
            H[norm]+=m
    m=0
    theta=-1
    for i,j in enumerate(H):
        if j>m:
            m=j
            theta=i
    return [theta,m]

"""
Sift Feature Desciptor
:param feat - list of interest points a.k.a. keypoints
:returns 
"""
def sift(img,feat):
    print("shape of img: "+str(img.shape))

    sift=[]
    tmp=[]
    for f in feat:
        if f[0]>8 and f[0]<img.shape[0]-8 and f[1]>8 and f[1]<img.shape[1]-8:
            tmp.append(f)
    feat=tmp
    for i in feat:
        r,c=i[0],i[1]
        A=np.zeros((4,4,2)) 
        for j in range(4):
            for k in range(4):
                A[j,k]=makeH(gk(6,6,1)*img[r-9+(j*4):r-3+(j*4),c-9+(k*4):c-3+(k*4)])
        B=np.zeros(8)
        for j in range(4):
            for k in range(4):
                B[int(A[j,k,0])]+=A[j,k,1]
        m=0
        theta=-1
        for i,j in enumerate(B):
            if j>m:
                m=j
                theta=i
        sift.append((theta,m))
    return sift


